# Задача 7 – графовые базы!

- Установить 2 графовые базы из списка DB-Engines Ranking
- Предпочтительные - neo4j, nebula, arangodb
- Предпочтительный язык запросов cypher
- Создать ipynb ноутбук в котором:
- Считать данные из источника https://disk.yandex.ru/d/s6wWqd8Ol_5IvQ
- Внести данные из таблицы в графовую БД
- Построить графовое представление в БД, осуществить несколько запросов на языке запросов к графовой БД
- Найти взаимосвязи визуально и с помощью алгоритмов (алгоритмы на ваше усмотрение)
- Написать rest сервис на python к графовой БД в котором на вход поступает ФИО, на выходе graphml или json
- Результаты представить на гитхаб и в виде кода + небольшой презентации

In [1]:
#!pip install py2neo
from py2neo import Graph

In [2]:
# python драйвер для Neo4j в обертке  py2neo https://py2neo.org/2021.1/

graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4j"))
graph.run("SHOW DATABASES")

name,type,aliases,access,address,role,writer,requestedStatus,currentStatus,statusMessage,default,home,constituents
neo4j,standard,[],read-write,localhost:7687,primary,true,online,online,,true,true,[]
system,system,[],read-write,localhost:7687,primary,true,online,online,,false,false,[]


In [3]:
#!pip3 install pymgclient
import mgclient

In [4]:
# #  python bolt драйвер для Memgraph

# conn = mgclient.connect(host='127.0.0.1', port=7687)

# # Create a cursor for query execution
# cursor = conn.cursor()

# # Execute a query
# cursor.execute("""
#         CREATE (n:Person {name: 'John'})-[e:KNOWS]->
#                (m:Person {name: 'Steve'})
#         RETURN n, e, m
#     """)

# # Fetch one row of query results
# row = cursor.fetchone()

# print(row[0])
# #(:Person {'name': 'John'})

# print(row[1])
# #[:KNOWS]

# print(row[2])
# #(:Person {'name': 'Steve'})

# # Make database changes persistent
# conn.commit()

## ELT

In [5]:
import pandas as pd
import numpy as np

In [6]:
# поправим столбцы при открытии

df = pd.read_csv('data_test.csv', delimiter=';', names = ['id_event', 'full_name_event1', 'full_name_event2'])
display(df.info())
display(df)
list(df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_event          5001 non-null   object
 1   full_name_event1  5001 non-null   object
 2   full_name_event2  5001 non-null   object
dtypes: object(3)
memory usage: 117.3+ KB


None

,id_event,full_name_event1,full_name_event2
0,id события,ФИО участника события 1,ФИО участника события 2
1,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
2,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
3,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
4,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
...,...,...,...
4996,999333,Осташов Владимир Данилович,Чалов Илья Владимирович
4997,999360,Гандыбина Любовь Александровна,Мерлин Илья Юрьевич
4998,999403,Востоков Виктор Ильдарович,Аликас Никита Андреевич
4999,999405,Огарева Людмила Ильдаровна,Нагайцева Алина Степановна


['id_event', 'full_name_event1', 'full_name_event2']

In [7]:
display(df['id_event'].value_counts())#.tolist())
display(df['full_name_event1'].value_counts())#.tolist())
display(df['full_name_event2'].value_counts())#.tolist())

613539    2
92995     2
390312    2
551592    2
358194    2
         ..
333694    1
333678    1
333635    1
333448    1
999878    1
Name: id_event, Length: 4986, dtype: int64

Ахромеева Алина Ивановна          49
Башнина Антонина Глебовна         14
Диомидов Игорь Ильдарович          5
Зимнухова Карина Даниловна         4
Пафомова Кира Вадимовна            2
                                  ..
Серафимина Екатерина Никитовна     1
Башмашников Тимур Рамилевич        1
Онищин Федор Аркадьевич            1
Речкалов Марсель Антонович         1
Ряполовский Георгий Петрович       1
Name: full_name_event1, Length: 4931, dtype: int64

Медведева Дарья Алексеевна          5
Двигубская Валентина Геннадьевна    2
Шолохов Игорь Робертович            2
ФИО участника события 2             1
Шлякова Маргарита Алексеевна        1
                                   ..
Фомич Семен Ильич                   1
Коковкина Валерия Дамировна         1
Токранов Данил Степанович           1
Аврелин Сергей Кириллович           1
Жилейкин Виктор Павлович            1
Name: full_name_event2, Length: 4995, dtype: int64

In [8]:
patt = "(?:{})".format(df["full_name_event2"].replace("", np.nan).dropna().str.lower().str.cat(sep="|"))
df["Result"] = df["full_name_event1"].str.lower().str.contains(patt)
display(df["Result"].value_counts())
df.to_csv('data_test1.csv', sep=';', encoding='UTF-8')

False    4923
True       78
Name: Result, dtype: int64

In [9]:
df1=df.loc[df['full_name_event1'] == 'Ахромеева Алина Ивановна']
print(list(df1.index))

[222, 276, 309, 321, 472, 477, 643, 690, 885, 912, 1007, 1145, 1222, 1611, 1696, 1750, 1861, 1937, 2092, 2148, 2223, 2286, 2447, 2651, 2839, 2845, 2958, 3080, 3153, 3171, 3176, 3265, 3295, 3621, 3650, 3766, 3882, 3912, 4087, 4091, 4113, 4151, 4202, 4243, 4252, 4257, 4275, 4299, 4393]


#### ВЫВОД id_event повторяются, также повторения в full_name_event1 и full_name_event2 учтем при разработке скрипта загрузки графа

In [10]:
# import csv
# with open('data_test1.csv', 'r', encoding='UTF-8') as csvfile:
#     reader = csv.reader(csvfile, delimiter=';')
#     temp_id= -2
#     #next(reader)
#     next(reader)
#     for row in reader:
#         if temp_id != row[1]:
#         #print(row)
#             print('CREATE (label'+row[0]+':Event {name:'+row[1]+', id_event:'+row[1]+'})')
#             print('CREATE (aIperson'+row[0]+':Person {name:"'+row[2]+'"})')
#             print('CREATE (aIIperson'+row[0]+':Person {name:"'+row[3]+'"})')
#             print('CREATE (aIperson'+row[0]+')-[:TAKE_PART]->(label'+row[0]+')')
#             print('CREATE (aIIperson'+row[0]+')-[:TAKE_PART]->(label'+row[0]+')')
#             temp_id = row[1]
#             #print(temp_id)
#         else:
#             temp_temp_id=str(int(row[0])-1)
#             #print('#CREATE (label'+row[0]+':Event {name:'+row[1]+', id_event:'+row[1]+'})')
#             print('#CREATE (aIperson'+row[0]+':Person {name:"'+row[2]+'"})')
#             print('#CREATE (aIIperson'+row[0]+':Person {name:"'+row[3]+'"})')
#             print('#CREATE (aIperson'+row[0]+')-[:TAKE_PART]->(label'+temp_temp_id+')')
#             print('#CREATE (aIIperson'+row[0]+')-[:TAKE_PART]->(label'+temp_temp_id+')')
#             print('#CREATE (aIperson'+row[0]+')-[:FAMILIAR]->(aIperson'+temp_temp_id+')')
#             print('#CREATE (aIIperson'+row[0]+')-[:FAMILIAR]->(aIIperson'+temp_temp_id+')')
#             temp_id = row[1]
#             #print(temp_id)
            
        

In [12]:
import csv
from contextlib import redirect_stdout
with open('data_test1.csv', 'r', encoding='UTF-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    temp_id= -2
    #next(reader)
    next(reader)
    with open('out.txt', 'w', encoding='UTF-8') as f:
        for row in reader:
        #print(row)
       
        #with open('out.txt', 'w') as f:
            with redirect_stdout(f):
                if temp_id != row[1]:
        #print(row)
                    print('CREATE (label'+row[0]+':Event {name:'+row[1]+', id_event:'+row[1]+'})')
                    print('CREATE (aIperson'+row[0]+':Person {name:"'+row[2]+'"})')
                    print('CREATE (aIIperson'+row[0]+':Person {name:"'+row[3]+'"})')
                    print('CREATE (aIperson'+row[0]+')-[:TAKE_PART]->(label'+row[0]+')')
                    print('CREATE (aIIperson'+row[0]+')-[:TAKE_PART]->(label'+row[0]+')')
                    temp_id = row[1]
            #print(temp_id)
                else:
                    temp_temp_id=str(int(row[0])-1)
            #print('#CREATE (label'+row[0]+':Event {name:'+row[1]+', id_event:'+row[1]+'})')
                    print('CREATE (aIperson'+row[0]+':Person {name:"'+row[2]+'"})')
                    print('CREATE (aIIperson'+row[0]+':Person {name:"'+row[3]+'"})')
                    print('CREATE (aIperson'+row[0]+')-[:TAKE_PART]->(label'+temp_temp_id+')')
                    print('CREATE (aIIperson'+row[0]+')-[:TAKE_PART]->(label'+temp_temp_id+')')
                    print('CREATE (aIperson'+row[0]+')-[:FAMILIAR]->(aIperson'+temp_temp_id+')')
                    print('CREATE (aIIperson'+row[0]+')-[:FAMILIAR]->(aIIperson'+temp_temp_id+')')
                    temp_id = row[1]
            #print(temp_id)

![](pic/2023-03-06_12-54-16.png)

In [13]:
def replacetext(str1, str2):
    with open("out.txt", "rt", encoding='utf-8') as file:
        x = file.read()

    with open("out.txt", "wt", encoding='utf-8') as file1:
        x = x.replace(str1,str2)
        file1.write(x)

plist = ['aIperson276', 'aIperson309', 'aIperson321', 'aIperson472', 'aIperson477', 'aIperson643', 'aIperson690', 'aIperson885', 'aIperson912', 'Iperson1007', 'aIperson1145', 'aIperson1222', 'aIperson1611', 'aIperson1696', 'aIperson1750', 'aIperson1861',
          'aIperson1937', 'aIperson2092', 'aIperson2148', 'aIperson2223', 'aIperson2286', 'aIperson2447', 'aIperson2651', 'aIperson2839', 'aIperson2845', 'aIperson2958', 'aIperson3080', 'aIperson3153', 'aIperson3171', 'aIperson3176', 'aIperson3265', 'aIperson3295', 
          'aIperson3621', 'aIperson3650', 'aIperson3766', 'aIperson3882', 'aIperson3912', 'aIperson4087', 'aIperson4091', 'aIperson4113', 'aIperson4151', 'aIperson4202', 'aIperson4243', 'aIperson4252', 'aIperson4257', 'aIperson4275', 'aIperson4299',
          'aIperson4393']

#print(plist)    
    
for i in plist:
    #print(i)
    replacetext(i, 'aIperson222')

In [14]:
#!pip install graphdatascience

In [15]:
from graphdatascience import GraphDataScience

host = "bolt://localhost:7687"
user = "neo4j"
password= "neo4j"

gds = GraphDataScience(host, auth=(user, password))
gds.set_database("neo4j")
print(gds.version())

2.3.1


In [75]:
gds.run_cypher(
  """
  CALL gds.graph.project(
  'myGraph3',
  'Person',
  {
    KNOWS: {
      orientation: 'UNDIRECTED'
    }
  }
)
"""
)

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Person': {'label': 'Person', 'properties': {}}}","{'KNOWS': {'orientation': 'UNDIRECTED', 'index...",myGraph3,10000,0,21


In [76]:
gds.run_cypher(
  """
CALL gds.wcc.stream('myGraph')
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).name AS name, componentId
ORDER BY componentId, name  
"""
)

,name,componentId
0,None,0
1,None,1
2,None,2
3,None,3
4,None,4
...,...,...
10001,None,10000
10002,None,10000
10003,None,10000
10004,None,10000


In [77]:
gds.run_cypher(
  """
CALL gds.labelPropagation.stream('myGraph')
YIELD nodeId, communityId AS Community
RETURN gds.util.asNode(nodeId).name AS Name, Community
ORDER BY Community, Name
  """
)

,Name,Community
0,None,1
1,None,2
2,None,4
3,None,5
4,None,7
...,...,...
10001,None,15001
10002,None,15001
10003,None,15001
10004,None,15001


In [78]:
gds.run_cypher(
  """
CALL gds.triangleCount.stream('myGraph')
YIELD nodeId, triangleCount
RETURN gds.util.asNode(nodeId).name AS name, triangleCount
ORDER BY triangleCount DESC
  """
)

,name,triangleCount
0,None,3
1,None,2
2,None,2
3,None,1
4,None,1
...,...,...
10001,None,0
10002,None,0
10003,None,0
10004,None,0


In [79]:
gds.run_cypher(
  """
CALL gds.localClusteringCoefficient.stream('myGraph')
YIELD nodeId, localClusteringCoefficient
RETURN gds.util.asNode(nodeId).name AS name, localClusteringCoefficient
ORDER BY localClusteringCoefficient DESC  """
)

,name,localClusteringCoefficient
0,None,1.000000
1,None,1.000000
2,None,0.666667
3,None,0.666667
4,None,0.300000
...,...,...
10001,None,0.000000
10002,None,0.000000
10003,None,0.000000
10004,None,0.000000


In [ ]:
import bottle
from bottle import request, response
from bottle import post, get, put, delete

@get('/')
def getIndex():
    return '''
    Введите, пример <b>Балынин Дмитрий Эрлихович</b>
    <form method="post" action='/fullname'>
    <label for="fullname">ФИО:</label>
    <input type="text" name="fullname" id="name" value="" />
    <input type="submit" value="submit" />
    </form></body></html>
    '''
    
@post('/fullname')
def getAll():
    return '1111'
 
bottle.run()

Bottle v0.13-dev server starting up (using WSGIRefServer())...
Listening on http://127.0.0.1:8080/
Hit Ctrl-C to quit.

